# libs

In [7]:
import sys
import os

src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [8]:
import logging
from src.core.llm_generators.specification import AnalyticsSpecGenerator
from src.core.llm_generators.airflow import AirflowDagGenerator
from src.core.llm_generators.dbt import DbtGenerator
from src.core.llm_generators.dashboards import MetabaseDashboardGenerator
from src.config.prompts import prompts
from src.config.settings import settings

In [9]:
# logging.basicConfig(level=logging.INFO, filemode="w", filename="exp_meteo.log")
logging.basicConfig(level=logging.INFO)

## 1 ШАГ: Описание / Генерация ТЗ

In [10]:
user_description = """
Необходимо построить аналитическую систему для мониторинга погодных условий в Москве. 
Предполагается, что аналитикой будут пользоваться синоптики.

Источником данных является OpenMeteo API.
Требуется ежедневно (ночью в период с 00:00 по 02:00) собирать информацию о показателях за прошедший день:
    * температуре воздуха (на уровне 2 метра над землей)
    * влажности (на уровне 2 метра над землей)
    * скорости ветра (10 метров)
    * осадках

На основе этих данных нужно рассчитывать среднесуточные значения, а также выявлять дни с экстремальными погодными условиями (например, сильный ветер или аномально высокая температура). 
Итоговый дашборд должен отображать динамику погодных параметров и выделять аномалии."
"""

### GigaChat 2 Pro

In [25]:
spec_gen = AnalyticsSpecGenerator(with_metrics=True)

INFO:SPECIFICATION:Используется just-ai/gigachat/GigaChat-2-Pro с температурой 0.5


In [26]:
spec_giga = spec_gen.generate_spec(user_description)

INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:SPECIFICATION:Токены: всего=1364, prompt=673, completion=691; Стоимость=$0.0000000000; Время=12.94 сек
INFO:SPECIFICATION:ТЗ извлечено из пользовательского описания
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:SPECIFICATION:Токены: всего=1289, prompt=648, completion=641; Стоимость=$0.0000000000; Время=12.25 сек
INFO:SPECIFICATION:Сгенерированы рекомендации
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря
INFO:SPECIFICATION:Извлеченное ТЗ сохранено в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/analytics_spec.yml


### YandexGPT Pro 5

In [27]:
spec_gen = AnalyticsSpecGenerator(with_metrics=True,
                                  model="just-ai/yandexgpt/yandexgpt/rc")

INFO:SPECIFICATION:Используется just-ai/yandexgpt/yandexgpt/rc с температурой 0.5


In [28]:
spec_yandex = spec_gen.generate_spec(user_description)

INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:SPECIFICATION:Токены: всего=1423, prompt=711, completion=712; Стоимость=$0.0000000000; Время=9.85 сек
INFO:SPECIFICATION:ТЗ извлечено из пользовательского описания
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:SPECIFICATION:Токены: всего=1385, prompt=630, completion=755; Стоимость=$0.0000000000; Время=9.93 сек
INFO:SPECIFICATION:Сгенерированы рекомендации
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря
INFO:SPECIFICATION:Извлеченное ТЗ сохранено в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/analytics_spec.yml


### Gemini

In [35]:
spec_gen = AnalyticsSpecGenerator(with_metrics=True,
                                  model="just-ai/gemini/gemini-2.0-flash")

INFO:SPECIFICATION:Используется just-ai/gemini/gemini-2.0-flash с температурой 0.5


In [ ]:
spec_gemini = spec_gen.generate_spec(user_description)

INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:SPECIFICATION:Токены: всего=1744, prompt=770, completion=974; Стоимость=$0.0000000000; Время=5.94 сек
INFO:SPECIFICATION:ТЗ извлечено из пользовательского описания
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:SPECIFICATION:Токены: всего=1980, prompt=822, completion=1158; Стоимость=$0.0000000000; Время=6.96 сек
INFO:SPECIFICATION:Сгенерированы рекомендации
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря
INFO:SPECIFICATION:Извлеченное ТЗ сохранено в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/analytics_spec.yml


In [35]:
(0.017 * (822+770) / 1000) + (0.066*(974+1158) / 1000)

0.167776

In [20]:
spec_t_pro = spec_gen._from_dict_to_analytics_spec(t)

INFO:SPECIFICATION:Объект AnalyticsSpec загружен из словаря


## 2 ШАГ: Генерация Airflow DAG

In [9]:
spec_gen = AnalyticsSpecGenerator(user_description)
spec = spec_gen._from_yml_to_analytics_spec("../artifacts/deploy_meteo/analytics_spec_gigachat.yml")

INFO:SPECIFICATION:Используется just-ai/gigachat/GigaChat-2-Pro с температурой 0.0
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из файла ../artifacts/deploy_meteo/analytics_spec_gigachat.yml


### Gpt-4o

In [10]:
airflow_gen = AirflowDagGenerator(analytics_specification=spec,
                                  with_metrics=True)
airflow_gen.generate_dag()

INFO:AIRFLOW:Используется just-ai/openai-proxy/gpt-4o с температурой 0.0
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=282, prompt=231, completion=51; Стоимость=0.179 ₽; Время=1.03 сек
INFO:AIRFLOW:Аргументы для DAG сгенерированы
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=5584, prompt=4784, completion=800; Стоимость=3.281 ₽; Время=8.39 сек
INFO:AIRFLOW:Функция moving_data_from_source_to_dwh сгенерирована
INFO:AIRFLOW:Шаблон DAG заполнен
INFO:AIRFLOW:DAG сохранен в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dags/weather_monitoring_moscow.py


### DeepSeek

In [11]:
airflow_gen = AirflowDagGenerator(analytics_specification=spec,
                                  with_metrics=True,
                                  model="just-ai/deepseek/deepseek-r1")
airflow_gen.generate_dag()

INFO:AIRFLOW:Используется just-ai/deepseek/deepseek-r1 с температурой 0.0
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=720, prompt=244, completion=476; Стоимость=0.750 ₽; Время=8.23 сек
INFO:AIRFLOW:Аргументы для DAG сгенерированы
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=6751, prompt=5015, completion=1736; Стоимость=4.799 ₽; Время=26.30 сек
INFO:AIRFLOW:Функция moving_data_from_source_to_dwh сгенерирована
INFO:AIRFLOW:Шаблон DAG заполнен
INFO:AIRFLOW:DAG сохранен в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dags/weather_in_moscow.py


### Claude

In [12]:
airflow_gen = AirflowDagGenerator(analytics_specification=spec,
                                  with_metrics=True,
                                  model="just-ai/claude/claude-sonnet-4")
airflow_gen.generate_dag()

INFO:AIRFLOW:Используется just-ai/claude/claude-sonnet-4 с температурой 0.0
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=370, prompt=303, completion=67; Стоимость=0.316 ₽; Время=2.99 сек
INFO:AIRFLOW:Аргументы для DAG сгенерированы
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=6443, prompt=5385, completion=1058; Стоимость=5.289 ₽; Время=16.95 сек
INFO:AIRFLOW:Функция moving_data_from_source_to_dwh сгенерирована
INFO:AIRFLOW:Шаблон DAG заполнен
INFO:AIRFLOW:DAG сохранен в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dags/weather_moscow_monitoring.py


### YandexGPT Pro 5

In [13]:
airflow_gen = AirflowDagGenerator(analytics_specification=spec,
                                  with_metrics=True,
                                  model="just-ai/yandexgpt/yandexgpt/rc")
airflow_gen.generate_dag()

INFO:AIRFLOW:Используется just-ai/yandexgpt/yandexgpt/rc с температурой 0.0
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=267, prompt=210, completion=57; Стоимость=0.160 ₽; Время=1.04 сек
INFO:AIRFLOW:Аргументы для DAG сгенерированы
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:AIRFLOW:Токены: всего=5700, prompt=5227, completion=473; Стоимость=3.420 ₽; Время=7.59 сек
INFO:AIRFLOW:Функция moving_data_from_source_to_dwh сгенерирована
INFO:AIRFLOW:Шаблон DAG заполнен
INFO:AIRFLOW:DAG сохранен в /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dags/Погода в Москве.py


## 3 ШАГ: Генерация DBT

In [6]:
spec_gen = AnalyticsSpecGenerator(user_description)
spec = spec_gen._from_yml_to_analytics_spec("../artifacts/analytics_spec_gigachat.yml")

INFO:SPECIFICATION:Используется just-ai/gigachat/GigaChat-2-Pro с температурой 0.0
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из файла ../artifacts/analytics_spec_gigachat.yml


### GPT-4o

In [8]:
dbt_llm = DbtGenerator(analytics_specification=spec,
                       with_metrics=True)
dbt_llm.generate_project()

INFO:DBT:Используется just-ai/openai-proxy/gpt-4o с температурой 0.0
INFO:root:sources.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/source.yml
INFO:root:profiles.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/profiles.yml
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=840, prompt=443, completion=397; Стоимость=0.837 ₽; Время=5.31 сек
INFO:root:Stage-модели сгенерированы
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/schema.yml
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/stg_openmeteo_api.sql
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=2190, prompt=720, completion

### Claude Opus 4

In [7]:
dbt_llm = DbtGenerator(analytics_specification=spec,
                       with_metrics=True,
                       model="just-ai/claude/claude-opus-4")
dbt_llm.generate_project()

INFO:DBT:Используется just-ai/claude/claude-opus-4 с температурой 0.0
INFO:root:sources.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/source.yml
INFO:root:profiles.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/profiles.yml
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=1532, prompt=551, completion=981; Стоимость=13.509 ₽; Время=17.24 сек
INFO:root:Stage-модели сгенерированы
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/schema.yml
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/stg_weather_metrics.sql
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=4413, prompt=1104, com

### GigaChat 2 Max

In [7]:
dbt_llm = DbtGenerator(analytics_specification=spec,
                       with_metrics=True,
                       model="just-ai/gigachat/GigaChat-2-Max")
dbt_llm.generate_project()

INFO:DBT:Используется just-ai/gigachat/GigaChat-2-Max с температурой 0.0
INFO:root:sources.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/source.yml
INFO:root:profiles.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/profiles.yml
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=512, prompt=30, completion=482; Стоимость=0.998 ₽; Время=9.25 сек
INFO:root:Stage-модели сгенерированы
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/schema.yml
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/stg_openmeteo_api.sql
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=1647, prompt=786, complet

### DeepSeek

In [9]:
dbt_llm = DbtGenerator(analytics_specification=spec,
                       with_metrics=True,
                       model="just-ai/deepseek/deepseek-r1")
dbt_llm.generate_project()

INFO:DBT:Используется just-ai/deepseek/deepseek-r1 с температурой 0.0
INFO:root:sources.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/source.yml
INFO:root:profiles.yml сгенерирован
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/profiles.yml
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=1376, prompt=462, completion=914; Стоимость=1.437 ₽; Время=17.72 сек
INFO:root:Stage-модели сгенерированы
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/schema.yml
INFO:DBT:Файл сохранен /home/ffrankusha/study/university/AI-assistant-for-DE/artifacts/deploy/dbt/models/stage/stg_open_meteo.sql
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DBT:Токены: всего=4423, prompt=835, completion

## 4 ШАГ: генерация дашборда

In [11]:
import yaml
with open("../artifacts/deploy/dbt/models/marts/schema.yml", "r", encoding="utf-8") as f:
    schema = yaml.safe_load(f)

In [12]:
spec_gen = AnalyticsSpecGenerator(user_description)
spec = spec_gen._from_yml_to_analytics_spec("../artifacts/analytics_spec_gigachat.yml")

metrics = spec.metrics

INFO:SPECIFICATION:Используется just-ai/gigachat/GigaChat-2-Pro с температурой 0.0
INFO:SPECIFICATION:Объект AnalyticsSpec загружен из файла ../artifacts/analytics_spec_gigachat.yml


### GPT-4o

In [7]:
dash_gen = MetabaseDashboardGenerator(
    metabase_url=settings.METABASE_URL,
    username=settings.METABASE_USERNAME,
    password=settings.METABASE_PASSWORD,
    with_metrics=True,
    model="just-ai/openai-proxy/gpt-4o"
)

dash_gen.generate_dashboard(marts_schema=schema,
                            metrics=metrics)

INFO:DASHBOARD:Аутентификация в Metabase прошла успешно!
INFO:DASHBOARD:Используется just-ai/openai-proxy/gpt-4o с температурой 0.0
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DASHBOARD:Токены: всего=2561, prompt=1959, completion=602; Стоимость=1.796 ₽; Время=7.18 сек
INFO:DASHBOARD:Сгенерированы настройки для 6 карточек
INFO:DASHBOARD:Карточка 'Среднесуточная температура' создана!
INFO:DASHBOARD:Карточка 'Среднесуточная влажность' создана!
INFO:DASHBOARD:Карточка 'Среднесуточная скорость ветра' создана!
INFO:DASHBOARD:Карточка 'Осадки' создана!
INFO:DASHBOARD:Карточка 'Температурные аномалии' создана!
INFO:DASHBOARD:Карточка 'Индекс комфорта' создана!
INFO:DASHBOARD:Дашборд 'Analytics Dashboard' создан!
INFO:DASHBOARD:Карточка №129 добавлена на дашборд №129
INFO:DASHBOARD:Карточка №130 добавлена на дашборд №129
INFO:DASHBOARD:Карточка №131 добавлена на дашборд №129
INFO:DASHBOARD:Карточка №132 добавлена на дашборд №129
INFO:

### Claude Opus 4

In [15]:
dash_gen = MetabaseDashboardGenerator(
    metabase_url=settings.METABASE_URL,
    username=settings.METABASE_USERNAME,
    password=settings.METABASE_PASSWORD,
    with_metrics=True,
    model="just-ai/claude/claude-opus-4"
)

dash_gen.generate_dashboard(marts_schema=schema,
                            metrics=metrics)

INFO:DASHBOARD:Аутентификация в Metabase прошла успешно!
INFO:DASHBOARD:Используется just-ai/claude/claude-opus-4 с температурой 0.0
INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DASHBOARD:Токены: всего=4507, prompt=2492, completion=2015; Стоимость=31.113 ₽; Время=36.68 сек
INFO:DASHBOARD:Сгенерированы настройки для 11 карточек
INFO:DASHBOARD:Карточка 'current_temperature' создана!
INFO:DASHBOARD:Карточка 'temperature_trend' создана!
INFO:DASHBOARD:Карточка 'humidity_trend' создана!
INFO:DASHBOARD:Карточка 'wind_speed_trend' создана!
INFO:DASHBOARD:Карточка 'precipitation_bars' создана!
INFO:DASHBOARD:Карточка 'temperature_anomalies' создана!
INFO:DASHBOARD:Карточка 'anomaly_severity_distribution' создана!
INFO:DASHBOARD:Карточка 'comfort_index_heatmap' создана!
INFO:DASHBOARD:Карточка 'daily_comfort_trend' создана!
INFO:DASHBOARD:Карточка 'alerts_summary' создана!
INFO:DASHBOARD:Карточка 'recent_alerts_table' создана!
INFO:DAS

### GigaChat 2 Pro

In [ ]:
dash_gen = MetabaseDashboardGenerator(
    metabase_url=settings.METABASE_URL,
    username=settings.METABASE_USERNAME,
    password=settings.METABASE_PASSWORD,
    with_metrics=True,
    model="just-ai/gigachat/GigaChat-2-Pro"
)
dash_gen.generate_dashboard(marts_schema=schema,
                            metrics=metrics)

### Gemini 2 Flash

In [13]:
dash_gen = MetabaseDashboardGenerator(
    metabase_url=settings.METABASE_URL,
    username=settings.METABASE_USERNAME,
    password=settings.METABASE_PASSWORD,
    with_metrics=True,
    model="just-ai/gemini/gemini-2.0-flash"
)

INFO:DASHBOARD:Аутентификация в Metabase прошла успешно!
INFO:DASHBOARD:Используется just-ai/gemini/gemini-2.0-flash с температурой 0.0


In [14]:
dash_gen.generate_dashboard(marts_schema=schema,
                            metrics=metrics)

INFO:httpx:HTTP Request: POST https://caila.io/api/adapters/openai/chat/completions "HTTP/1.1 200 "
INFO:DASHBOARD:Токены: всего=3053, prompt=2103, completion=950; Стоимость=0.098 ₽; Время=5.37 сек
INFO:DASHBOARD:Сгенерированы настройки для 12 карточек
INFO:DASHBOARD:Карточка 'Avg Daily Temperature' создана!
INFO:DASHBOARD:Карточка 'Avg Daily Humidity' создана!
INFO:DASHBOARD:Карточка 'Avg Daily Wind Speed' создана!
INFO:DASHBOARD:Карточка 'Total Daily Precipitation' создана!
INFO:DASHBOARD:Карточка 'Temperature Anomalies' создана!
INFO:DASHBOARD:Карточка 'Average Comfort Index' создана!
INFO:DASHBOARD:Карточка 'High Temperature Alerts' создана!
INFO:DASHBOARD:Карточка 'High Humidity Alerts' создана!
INFO:DASHBOARD:Карточка 'High Wind Alerts' создана!
INFO:DASHBOARD:Карточка 'High Precipitation Alerts' создана!
INFO:DASHBOARD:Карточка 'Significant Anomaly Alerts' создана!
INFO:DASHBOARD:Карточка 'Low Comfort Alerts' создана!
INFO:DASHBOARD:Дашборд 'Analytics Dashboard' создан!
INFO:DAS